In [1]:
import os
from pathlib import Path

# Find the portfolio_projects directory by searching up the directory tree
current_dir = Path.cwd()
project_root = None

# Search up from current directory for portfolio_projects
for parent in [current_dir] + list(current_dir.parents):
    if (parent / "config" / "config.yaml").exists() and (parent / "src").exists():
        project_root = parent
        break

# If not found, try a known path
if project_root is None:
    project_root = Path(__file__).parent.parent if hasattr(__builtins__, '__file__') else current_dir
    # Fallback: navigate to portfolio_projects from the notebook location
    if "research" in str(current_dir):
        project_root = current_dir.parent
    elif "portfolio_projects" not in str(current_dir):
        project_root = current_dir / "portfolio_projects"

os.chdir(project_root)

# Verify the project root
print(f"Working directory: {os.getcwd()}")
print(f"Config file exists: {(project_root / 'config' / 'config.yaml').exists()}")
print(f"Src folder exists: {(project_root / 'src').exists()}")


Working directory: c:\Users\asidd\Desktop\Data_science_projects\MLOps\portfolio_projects
Config file exists: True
Src folder exists: True


In [2]:
import os
import sys
from dataclasses import dataclass
from pathlib import Path

# Add src to path so we can import portfolio_projects
sys.path.insert(0, str(Path.cwd() / "src"))

# Now import from portfolio_projects
from portfolio_projects.utils.common import read_yaml, create_directories
from portfolio_projects.logger import logger
from portfolio_projects.utils.common import get_size
import zipfile
import gdown

# Define config file paths as absolute paths
project_root = Path.cwd()
CONFIG_FILE_PATH = project_root / "config" / "config.yaml"
PARAMS_FILE_PATH = project_root / "params.yaml"


In [3]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [4]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config


In [5]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix+file_id, str(zip_download_dir))
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e


    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)     


In [6]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    


[2026-02-10 14:33:35,712: INFO: common: created directory at: artifacts]
[2026-02-10 14:33:35,723: INFO: common: created directory at: artifacts/data_ingestion]
[2026-02-10 14:33:35,726: INFO: 675195999: Downloading data from https://drive.google.com/file/d/1vWDeW9_Ew96WyTYgpOpQcFrHFdsSFOZj/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1vWDeW9_Ew96WyTYgpOpQcFrHFdsSFOZj
From (redirected): https://drive.google.com/uc?export=download&id=1vWDeW9_Ew96WyTYgpOpQcFrHFdsSFOZj&confirm=t&uuid=90753379-38d9-48d6-8d3c-9c8e502fc5c9
To: c:\Users\asidd\Desktop\Data_science_projects\MLOps\portfolio_projects\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [00:09<00:00, 12.7MB/s] 


[2026-02-10 14:33:51,418: INFO: 675195999: Downloaded data from https://drive.google.com/file/d/1vWDeW9_Ew96WyTYgpOpQcFrHFdsSFOZj/view?usp=sharing into file artifacts/data_ingestion/data.zip]
